In [ ]:
import spacy
import re


In [ ]:
nlp = spacy.load('en_core_web_sm')

def process_text_spacy(text):

    doc = nlp(text)
    
    processed_sentences = []

    for sent in doc.sents:
        print(f"ORIGINAL: {sent.text.strip()}")
        

        lemmatized_words = [
            token.lemma_.lower() for token in sent 
            if not token.is_stop and token.is_alpha
        ]

        if lemmatized_words:
            processed_sentences.append(lemmatized_words)
        
        print(f"LEMMATIZED: {lemmatized_words}")
        print("#----\n\n")

    return processed_sentences


In [ ]:

stopwords_set = set(nltk.corpus.stopwords.words('english'))

wordcloud = WordCloud(
    stopwords=stopwords_set,
    background_color="white",
    width=1600,
    height=800,
    max_words=200,
    colormap='magma',
    collocations=False
).generate(lemmatized_text_for_cloud)

plt.figure(figsize=(20, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Termos Lematizados Mais Frequentes nas Atas do COPOM', fontsize=20, pad=20)
plt.tight_layout()
plt.show()